# BCxP GenAI Hands-on Sessions

In diesem Jupyter Notebook finden sich fünf Hands-On Session zur Verwendung von generativer KI in Software Anwendungen. 

1. OpenAI API - Basics
2. OpenAI API - Function Calling
3. Semantic Search & RAG
4. OpenAI API - Working with Files
5. Generative UI

Es empfiehlt sich mit Part 1 zu beginnen. Die weiteren Teile bauen nicht aufeinander auf und können in beliebeger Reihenfolge bearbeitet werden. 

Thematisch drehen sich die Sessions um die Entwicklung eines digitalen Kellners für ein Pizzeria.

## Setup

In [ ]:
# Install required packages
%pip install openai sentence-transformers matplotlib scikit-learn umap-learn plotly faiss-cpu numpy tabulate pandas

In [ ]:
# Import necessary libraries
from openai import OpenAI
import json
import faiss
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict
import io

from hack_helpers import OPENAI_API_KEY
from menu import MENU

# Initialize OpenAI client
openai_client = OpenAI(api_key= "")

# Part 1: OpenAI API Basics

## Finde die passende Pizza

Task: Ein Kunde soll basierend auf der Beschreibung einer Speise eine passende Bestelloption aus den verfügbaren Speisen (definiert in menu.py) finden können. 

## Verwendung von OpenAI API **ohne Structured Outputs**:


Führe die nächste Zelle aus und beobachte die Ausgabe. Die Ausgabe ist Textbasiert - wie bei der Nutzung von ChatGPT über die Web- oder die Desktop App. Eine programmatische Verwendung des Outputs ist mit dieser 'unstrukturierten' Ausgabe ungeeignet. 

**Aufgabe:** Führe die nächste Zelle mehrmals aus und verändere den User-Input. Beobachte, wie sich das Format der Ausgabe immer wieder verändert.

**Aufgabe:** Passe den System-Prompt an, um die Struktur der Ausgabe vorzugegeben (Beispielsweise "Gib den Namen genau einer Speise an"). Versuche dann den System-Prompt zu 'hacken' und mit einem beliebigen User-Input eine Ausgabe zu erhalten, die nicht der geforderten Struktur entspricht. Ist dies möglich? 


In [ ]:
# 🥡 Menü
# for item in MENU:
#     print(item)

# 💬 Beispiel-Eingabe
user_input = "Ich möchte gerne etwas mit scharfer Salami oder Pilzen."

# 🧠 API-Aufruf
response = openai_client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "developer", "content": "Du bist ein digitaler Kellner. Wähle den passenden Eintrag aus der Speisekarte aus."},
        {"role": "user", "content": f"Speisekarte:\n{json.dumps(MENU, ensure_ascii=False)}\n\nBestellung: {user_input}"} # Hier übergeben wir die Speisekarte und die Bestellung" 
    ],
)

# 📤 Ausgabe anzeigen
print(response.output_text)


## Verwendung von OpenAI API **mit Structured Outputs**:

Mit _structured outputs_ kann sichergestellt werden, dass das Format der Ausgabe immer dem selben JSON Schema entspricht. JSON (JavaScript Object Notation) ist ein leichtgewichtiges, textbasiertes Format zur Darstellung strukturierter Daten. Es wird häufig in der Webentwicklung und bei APIs zur Übertragung von Daten zwischen Client und Server verwendet. 

**Aufgabe:** Mache Anpassungen, damit nur eine Speise für die Bestellung empfohlen wird. Welche Möglichkeiten gibt es dies zu realisieren (passe das Prompt oder das Schema an). Welche Vorteile/Nachteile gibt es jeweils bei den unterschiedlichen Ansätzen? Kannst du den 'Prompt-Hacking' Ansatz aus der vorherigen Aufgabe auch unter der Verwendung von _structured outputs_ reproduzieren? 

In [ ]:
# 💬 Beispiel-Eingabe
# user_input = "Ich habe Lust auf eine würzige Pizza mit Salami." # Input mit einer Option
user_input = "Ich hätte gerne etwas mit scharfer Salami oder Peperoni." # Input mit mehreren Optionen

# JSON Schema für Ausgabe: Liste von Speisen, wobei eine Speise aus Id, Name, Beschreibung, Kategorie und Preis bestehen.
menu_items_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

# 🧠 API-Aufruf
response = openai_client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "developer", "content": "Du bist ein digitaler Kellner. Wähle den passenden Eintrag aus der Speisekarte aus."},
        {"role": "user", "content": f"Speisekarte:\n{json.dumps(MENU, ensure_ascii=False)}\n\nBestellung: {user_input}"}
    ],
    # Gewünschtes Ausgabeformat wird API-Aufruff übergeben
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": menu_items_schema,
            "strict": True
        }
    }
)
recommendations = json.loads(response.output_text)["items"]

# 📤 Nun können wir über Speiseempfehlung iterieren und diese beispielsweise Schritt für Schritt Anzeigen
for item in recommendations:
    print(item)


# Part 2: OpenAI API Function Calling

## Pizza Buchungssystem

Task: Der digitale Kellner soll Bestellungen in einer Bestellapp registrieren können. Die Bestellapp stellt dazu eine Funktion "add_item_to_order" bereit. Diese bekommt als Parameter die Id eines Menueintrags übergeben und fügt den Menüeintrag dann zu einer Liste hinzu.  

ChatGPT unterstützt sogenanntes Function Calling. Dabei erhält das Modell eine Beschreibung verfügbarer Funktionen inklusive ihrer Argumente. Basierend auf dem Nutzereingaben kann ChatGPT passende Funktionsaufrufe vorschlagen, die anschließend lokal ausgeführt werden können.

Zwar lässt sich eine vergleichbare Funktionalität grundsätzlich auch über strukturierte Ausgaben realisieren, jedoch stellt Function Calling eine einfachere und standardisierte Methode dafür dar.

**Aufgabe:** Erweitere die Funktionalität um eine Funktion "remove_item_from_order", welche die id eines bestelltem Items übergeben bekommt und dieses aus der Liste löscht. 

In [ ]:
from typing import List, Dict

# Bestellung (als globale Liste)
bestellung: List[Dict] = []

def add_item_to_order(item_id: str):
    item = next((entry for entry in MENU if entry["id"] == item_id), None)
    if item:
        bestellung.append(item)
        return f"{item['name']} wurde zur Bestellung hinzugefügt."
    return f"Item mit ID {item_id} nicht gefunden."

In [ ]:
tools = [
    {
        "type": "function",
        "name": "add_item_to_order",
        "description": "Fügt ein Item der Bestellung hinzu.",
        "parameters": {
            "type": "object",
            "properties": {
                "item_id": {
                    "type": "string",
                    "description": "Die ID des hinzuzufügenden Items.",
                }
            },
            "required": ["item_id"],
        },
    },
]

In [ ]:
def chat_with_waiter(user_input: str):
    print(f"👤 Input: {user_input}")
    # Erstellung der Eingabenachrichten
    input_messages = [
        {
            "role": "developer",
            "content": "Du bist ein digitaler Kellner. Nimm Bestellungen auf und beantworte Fragen zum Menü.",
        },
        {
            "role": "developer",
            "content": f"Speisekarte:\n{json.dumps(MENU, ensure_ascii=False)}",
        },
        {
            "role": "user",
            "content": user_input,
        },

    ]

    # API-Aufruf mit Function Calling
    response = client.responses.create(
        model="gpt-4o", input=input_messages, tools=tools, tool_choice="auto"
    )

    for tool_call in response.output:
        if tool_call.type != "function_call":
            print(f"🤖 Antwort: {response.output_text}")
            continue

        name = tool_call.name
        args = json.loads(tool_call.arguments)

        if name == "add_item_to_order":
            result = add_item_to_order(args["item_id"])
        print(f"🔧 Function Call: {name}({args}) ➜ {result}")

    # Anzeige der aktuellen Bestellung
    print("📦 Aktuelle Bestellung:")
    for item in bestellung:
        print(f"- {item['name']} ({item['preis']} €)")
    print('-' * 40+'\n')

In [ ]:
chat_with_waiter("Wie geht es dir? ")
chat_with_waiter("Ich möchte gerne eine Pizza Diavola und eine Cola bestellen.")
chat_with_waiter("Noch eine Cola bitte")
chat_with_waiter("Bitte entferne die Cola wieder.")

# Part 3: Semantic Search & Retrieval Augmented Generation (RAG)

## Embeddings & Basic Similarity Search

In der Einführung zu LLMs haben wir gesehen, wie Begriffe in einen Vektorraum eingebettet (embedded) und numerisch dargestellt werden können. Der Vektor eines Wortes - das **Embedding** - repräsentiert dabei dessen Bedeutung. 
Bei einer **syntaktischen** Suche nach dem Begriff "Unterkunft" in einem Dokument, werden genau diejenigen Wörter gesucht, die Zeichen für Zeichen mit "Unterkunft" übereinstimmen. Bei der **semantischen** Suche werden hingegen alle Wörter gesucht, die eine ähnliche Bedeutung wie "Unterkunft" besitzen. Das sind diejenigen Wörter, die im Vektorraum 'nahe' am Vektor von "Unterkunft" liegen. Neben Unterkunft würden so auch Begriffe wie "Ferienwohnung" oder "Hotel" auftauchen. Nachfolgend werden wir sehen, wie eine semantische Suche implementiert werden kann. Da der Vektorraum ein kontinuierlicher Raum ist werden bei der semantischen Suche entweder die top-k Ergebnisse verwendet die am nächsten sind oder es wird ein fester Schwellenwert verwendet, der definiert ob ein Kandidat nah genug am gesuchten Begriff ist um als 'Match' zu gelten.

Um Embeddings zu erstellen können unterschiedliche Sprachmodelle verwendet werden. Beispielsweise sind _SentenceTransformer_ eine ressourcen-schonende und frei verfügbare Alternative zu kostenpflichtigen Modellen wie von OpenAI, die auch lokal laufen können. Der einfachheit halber verwenden wir hier aber die OpenAI API um Embeddings zu erzeugen.

UMAP (Uniform Manifold Approximation and Projection) ist ein Verfahren zur Dimensionsreduktion, das hochdimensionale Vektoren – z. B. Embeddings aus Sprache – in einen niederdimensionalen Raum wie 2D für Darstellungszwecke abbildet. Es versucht dabei, semantische Nähe (z. B. zwischen Texten) möglichst gut zu bewahren, indem es lokale Nachbarschaften erhält.

⚠️ Wichtig: Die sichtbare Distanz im 2D-Plot entspricht nicht exakt der tatsächlichen Ähnlichkeit im ursprünglichen Vektorraum. Die Projektion ist eine visuelle Annäherung, keine metrische 1:1-Abbildung.

Führe die nächste Zelle aus um für jeden Menü Eintrag die Nähe zur User-Eingabe zu bestimmen und um die Embeddings der Speisen aus der Speisekarte und den User Input mittels UMAP in einem Plot zu visualisieren.

## Finde die passende Pizza mit Semantic Search

Task: Ein Kunde soll basierend auf der Beschreibung einer Speise eine passende Bestelloption finden können. 
**Aufgabe:** Füge der Karte einige Getränke und Salate hinzu und beobachte wie sich diese in die UMAP Projektion einfügen. Füge unpassende Nutzer-Eingaben ein und beobacht den Score und wie sich die Anfrage in die UMAP Projektion einfügt.

In [ ]:
# 🗣 Benutzereingabe
user_input = "Ich habe Lust auf eine spicy Pizza."

# 🧾 Strings zur Embedding-Vorbereitung
menu_texts = [f"{item['name']} – {item['beschreibung']}" for item in MENU]
texts_to_embed = [user_input] + menu_texts  # User prompt + all items

# 🧠 Embedding-Berechnung mit OpenAI
response = openai_client.embeddings.create(
    model="text-embedding-3-small", input=texts_to_embed
)
embeddings = [np.array(d.embedding) for d in response.data]

# 🧮 Vektorvergleich (cosine similarity)
user_embedding = embeddings[0] # User Prompt embedding
menu_embeddings = embeddings[1:] # Menu embeddings
# Hier wird die Cosine Similarity zwischen dem User-Embedding und den Menü-Embeddings berechnet
# Für viele Menü-Items und wechselnden Anfragen kann dies sehr ineffizient sein
similarities = cosine_similarity([user_embedding], menu_embeddings)[0] 

# 🥇 Sortierte Ergebnisse
results = []
for idx, score in sorted(enumerate(similarities), key=lambda x: x[1], reverse=True):
    item = MENU[idx]
    results.append(
        {
            "name": item["name"],
            "score": round(score, 3),
            "description": item["beschreibung"],
        }
    )

# 📊 Ausgabe als DataFrame
df = pd.DataFrame(results)
print(df)

plot_umap(user_embedding, menu_embeddings, MENU, similarity_scores=similarities)


## FAISS für schnelle Similarity Search bei großen Datenmengen

**FAISS** (Facebook AI Similarity Search) ist eine Bibliothek für schnelle und skalierbare ähnlichkeitssuche auf Vektoren.
Sie ermöglicht eine effiziente Nearest-Neighbor-Suche — nützlich für semantische Suche, Reccomendation Systems, Clustering usw.

Hauptmerkmale:
- Funktioniert mit hochdimensionalen Embeddings (z. B. von SentenceTransformers)
- Ermöglicht Kosinus-Ähnlichkeit (über normalisierte Vektoren und inneres Produkt) für semantische Ähnlichkeit
- Skaliert auf Millionen oder Milliarden von Vektoren
- Indizes können auf der Festplatte gespeichert und geladen werden – Metadaten müssen jedoch separat verwaltet werden

Typischer Workflow:
1.	Wandle deine Objekte (z. B. Texte) mit einem Embedding-Modell in dichte Vektoren um.
2.	Speichere diese Vektoren in einem FAISS-Index (z. B. IndexFlatIP oder IndexHNSWFlat).
3.	Kodierst du eine Anfrage (Query), kannst du die top-k ähnlichsten Objekte suchen.

**Aufgabe 1**: Finde ein Beispiel für eine User Query, für welches das Ergebnis mit dem besten Ähnlichkeitsscore nicht zur Query passt.

In [ ]:
# === EMBEDDING SETUP ===
# Wir wollen Embeddings erzeugen für Name + Beschreibung der Menüpunkte
menu_texts = [f"{item['name']}: {item['beschreibung']}" for item in MENU]

# Generiere die Embeddings für die Menütexte mit OpenAI
response = openai_client.embeddings.create(model="text-embedding-3-small", input=menu_texts)
embeddings = np.array([d.embedding for d in response.data], dtype=np.float32)

# === FAISS VECTOR INDEX ===
# Normalisierung wird benötigt um zusammen mit inner product (dot product) die Cosine Similarity zu berechnen, welche die Ähnlichkeit der Vektoren beschreibt
faiss.normalize_L2(embeddings)

dim = embeddings.shape[1] # Dimension der Vektoren
index = faiss.IndexFlatIP(dim) # Initialisierung des Indexes mit inner product (dot product)
index.add(embeddings) # Vektoren dem Index hinzufügen

# === MENÜ ABFRAGE ===
def retrieve_menu_items(query: str, top_k: int = 10) -> List[Dict]:
    # Step 1: Generiere Embedding für die User Abfrage
    response = openai_client.embeddings.create(
        model="text-embedding-3-small", input=[query]
    )
    query_vec = np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)

    # Step 2: Normalisiere den Vektor für die Cosine Similarity
    faiss.normalize_L2(query_vec)

    # Step 3: Suche im FAISS Index nach den ähnlichsten Vektoren
    distances, indices = index.search(query_vec, top_k)

    # Step 4: Erstelle eine Liste von Menüpunkten mit den entsprechenden Ähnlichkeitswerten
    return list(zip(indices[0], distances[0]))

print("FAISS Index Description")
print("-" * 30)
print(f"Index Type      : {type(index).__name__}")
print(f"Dimension (D)   : {index.d}")
print(f"Vectors Stored  : {index.ntotal}")
print("-" * 30 + "\n")

In [ ]:
# === EXAMPLE USAGE ===
user_query = "Ich hätte gerne etwas mit Pilzen"
retrieved_results = retrieve_menu_items(user_query)
for idx, score in retrieved_results:
    print(f"- {MENU[idx]['name']} (score: {score}): {MENU[idx]['beschreibung']}")

## Retrieval Augmented Generation (RAG) mit FAISS und OpenAI API

RAG (Retrieval-Augmented Generation) ist ein KI-Ansatz, der generative Sprachmodelle (z. B. GPT) mit einer externen Wissensquelle kombiniert. Dabei werden relevante Informationen zur Benutzeranfrage zuerst aus einer Datenbank oder Dokumentensammlung _retrieved_ (abgerufen), wie in folgendem Beispiel aus unserem FAISS Index, und anschließend von einem genertiven Modell in die Generierung der Antwort einbezogen. 

- Ein RAG Ansatz kann verwendet werden, wenn auf einem großen Datensatz gearbeitet werden muss, welcher zu groß für den Kontext des Sprachmodells ist.
- Außerdem können durch die Reduzierung des LLM Inputs auf relevante Einträge Kosten gesenkt werden (Anzahl der Input-Tokens sinkt).

**Aufgabe:** Überlege dir Vor und Nachteile eines RAG Ansatzes. In welchen Fällen kann der RAG Ansatz zu schlechten Ergebnissen führen? 

In [ ]:
menu_items_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

user_query = "Ich hätte gerne etwas mit Pilzen aber ohne Trüffel."
retrieved_results = retrieve_menu_items(user_query)
print("Vorselektierte Einträge aus dem Menü:\n")
for idx, score in retrieved_results:
    print(f"- {MENU[idx]['name']} (score: {score}): {MENU[idx]['beschreibung']}")
retrieved_menu_items = [MENU[idx] for idx, _ in retrieved_results]

# 🧠 API-Aufruf
response = openai_client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "system",
            "content": f"Du bist ein digitaler Kellner. Wähle genau eine passende Speise aus den folgenden vorselektierten Speisen aus, welche die Kriterien aus der Kundenbestellung am ehesten erfüllt:\n{json.dumps(retrieved_menu_items, ensure_ascii=False)}\n",
        },
        {
            "role": "user",
            "content": f"{user_query}",
        },
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": menu_items_schema,
            "strict": True,
        }
    },
)
recommendations = json.loads(response.output_text)["items"]

# 📤 Ausgabe anzeigen
print("\nEmpfohlene Speise:")
for item in recommendations:
    print(item)

## Local Embedding Generation (Bonus)

Nachfolgend findet sich ein Beispiel mit der Verwendung eines lokalen Sentence-Transformer Modells für das Erstellen der Embeddings der Speisekarte und des User-Inputs

In [ ]:
from hack_helpers import plot_umap
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# 🗣 2. Benutzereingabe
user_input = "Ich habe Lust auf eine spicy Pizza."

# 🧠 3. Modell laden (z. B. multilingual + performant)
model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# 🧮 4. Embeddings berechnen
menu_texts = [f"{item['name']} – {item['beschreibung']}" for item in MENU] # Hier wird die Speisekarte in emeddable Strings umgewandelt
menu_embeddings = model.encode(menu_texts, convert_to_tensor=True) # Hier werden die Embeddings für die gesamte Speisekarte berechnet
user_embedding = model.encode(user_input, convert_to_tensor=True) # Hier wird das Embedding für die Benutzereingabe berechnet

# TODO Print ein Embedding um die Vectorrepräsentation zu sehen
# print(user_embedding)

# 🔎 5. Ähnlichkeit berechnen
cos_scores = util.cos_sim(user_embedding, menu_embeddings)[0] # Hier wird die Ähnlichkeit zwischen der Benutzereingabe und den Speisen mittels cosine similarity berechnet

# 🥇 6. Sorted results
top_results = sorted(list(enumerate(cos_scores)), key=lambda x: x[1], reverse=True)

# 📤 7. Ausgabe strukturieren
results = []
for idx, score in top_results:
    item = MENU[idx]
    results.append(
        {
            "name": item["name"],
            "score": round(score.item(), 3),
            "description": item["beschreibung"],
        }
    )

# 📊 Als DataFrame anzeigen
pd.set_option("display.width", 200)  # Increase pd width to print correctly
df = pd.DataFrame(results)
print(df)
plot_umap(user_embedding, menu_embeddings, MENU, similarity_scores=cos_scores)

# Part 4: Working with Files

## Finde die passende Pizza

Task: Im Folgenden Szenario wird die Speisekarte von einer exteren Agentur erstellt und ist nur als PDF verfügbar. Im folgenden passen wir unseren digitalen Kellner so an, dass er mit der PDF Speisekarte arbeitet. 

## Verwendung von OpenAI API mit **Structured Outputs** und **Files** Input:

Multimodale Modelle sind KI-Modelle, die Informationen aus verschiedenen Modalitäten – etwa Text, Bilder, Audio oder strukturierte Daten – gleichzeitig verarbeiten und kombinieren können. Das unterscheidet sie von klassischen Modellen, die nur mit einer einzigen Eingabemodalität (z. B. nur Text) umgehen können.

GPT 4.1. beispielsweise kann multimodalen Input wie Text, PDFs oder Bilder verarbeiten. Im folgenden Beispiel verwenden wir `gpt-4.1` zur Verarbeitung eines Nutzer-Inputs *und* einer PDF-Datei (z. B. einer Speisekarte).

**Aufgabe:** Neben Speisen für die Pizzeria soll der Kellner auch Speisen für das Restaurant "Naherholungsgebiet" in Stuttgart aufnehmen können. Erweitere den API Aufruf, sodass beide Speisekarten übergeben werden. Füge im Systemprompt den aktuellen Tag hinzu und mache Anpassungen, dass nur die für den Tag verfügbaren Speisen des Naherholungsgebiets berücksichtigt werden. Erweitere das Output-Schema so, dass auch das entsprechende Restaurant ("Naherholungsgebite" oder "Salve") in der Ausgabe enthalten sind. Finde Bestellungen für die Speisen aus beiden Restaurants empfohlen werden. Passe den Prompt wo nötig an. 

In [ ]:
user_input = "Ich habe Lust auf was mit Gorgonzola."

with open("speisekarte.pdf", "rb") as f:
    data = f.read()
speisekarte_base64_string = base64.b64encode(data).decode("utf-8")

menu_items_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

# 🧠 API-Aufruf
response = openai_client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "developer",
            "content": [
                {
                    "type": "input_text",
                    "text": "Du bist ein digitaler Kellner. Wähle passende Einträge aus der Speisekarte.",
                },
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "filename": "speisekarte.pdf",
                    "file_data": f"data:application/pdf;base64,{speisekarte_base64_string}",
                },
                {
                    "type": "input_text",
                    "text": f"{user_input}",
                },
            ],
        }
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": menu_items_schema,
            "strict": True,
        }
    },
)

recommendations = json.loads(response.output_text)["items"]

# 📤 Ausgabe anzeigen
for item in recommendations:
    print(item)

## Part 5: Generative UI

Generative UI bezeichnet eine Benutzerschnittstelle, die sich dynamisch und kontextabhängig durch generative KI-Modelle anpassen oder erzeugen lässt 

### Beispiel
Ein Nutzer fragt:
„Zeig mir meine Verkaufszahlen der letzten 3 Monate im Vergleich zu Q1 2024.“
→ Die generative UI erstellt automatisch ein Dashboard mit dem passenden Plot, Filter und Interpretation – ohne dass das Layout oder die Abfrage vorher explizit programmiert war.

Im Folgenden wollen wir dem Nutzer ermöglichen Plots zu prompten und damit eine einfache Möglichkeit bieten den bereitgestellten Datensatz von Pizzaverkaufszahlen zu analysieren.

**Aufgabe:** Optimiere den System-Prompt, um fehlerhaften Code im Output zu vermeiden (General Task Description, Output Rules). Versuche alle Beispiele zum Laufen zu bringen! 

In [ ]:
# Lade Pizzaverkäufe-Daten aus einer CSV-Datei aus GitHub
url = "https://raw.githubusercontent.com/santoshkr23/Pizzasales/main/pizza_sales_dataset.csv"
df = pd.read_csv(url)

# Zeige Informationen über den DataFrame an 
df.info()
# Zeige die ersten 5 Zeilen des DataFrames an
df.head()

In [ ]:
openai_client = OpenAI(
    api_key=OPENAI_API_KEY
)

system_prompt = """
# General Task Description
Your task is to write complete Python code that - based on the provided pandas DataFrame (df) that includes pizza sales data and a user-specified plot request - produces a corresponding plotly.graph_objects.Figure.
The code must:
	1.	Transform the provided df through filtering, grouping, aggregating, or other methods to prepare the data for visualization.
	2.	Generate a well-styled plotly.graph_objects.Figure that fully satisfies the user's description and uses the transformed data.

# Output Rules (must be followed exactly)
    - ...
    

# Data
## df.info() output
    <class 'pandas.core.frame.DataFrame'>
    RangeIndex: 48620 entries, 0 to 48619
    Data columns (total 12 columns):
    #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
    0   pizza_id           48620 non-null  int64  
    1   order_id           48620 non-null  int64  
    2   pizza_name_id      48620 non-null  object 
    3   quantity           48620 non-null  int64  
    4   order_date         48620 non-null  object 
    5   order_time         48620 non-null  object 
    6   unit_price         48620 non-null  float64
    7   total_price        48620 non-null  float64
    8   pizza_size         48620 non-null  object 
    9   pizza_category     48620 non-null  object 
    10  pizza_ingredients  48620 non-null  object 
    11  pizza_name         48620 non-null  object 
    dtypes: float64(2), int64(3), object(7)
    memory usage: 4.5+ MB

## df.head() output
    pizza_id	order_id	pizza_name_id	quantity	order_date	order_time	unit_price	total_price	pizza_size	pizza_category	pizza_ingredients	pizza_name
    0	1	1	hawaiian_m	1	01-01-2015	11:38:36	13.25	13.25	M	Classic	Sliced Ham, Pineapple, Mozzarella Cheese	The Hawaiian Pizza
    1	2	2	classic_dlx_m	1	01-01-2015	11:57:40	16.00	16.00	M	Classic	Pepperoni, Mushrooms, Red Onions, Red Peppers,...	The Classic Deluxe Pizza
    2	3	2	five_cheese_l	1	01-01-2015	11:57:40	18.50	18.50	L	Veggie	Mozzarella Cheese, Provolone Cheese, Smoked Go...	The Five Cheese Pizza
    3	4	2	ital_supr_l	1	01-01-2015	11:57:40	20.75	20.75	L	Supreme	Calabrese Salami, Capocollo, Tomatoes, Red Oni...	The Italian Supreme Pizza
    4	5	2	mexicana_m	1	01-01-2015	11:57:40	16.00	16.00	M	Veggie	Tomatoes, Red Peppers, Jalapeno Peppers, Red O...	The Mexicana Pizza


# Few Shot Examples

## Example 1: 
    <input>
        Show a bar plot of the 5 worst sold pizzas based on pizza name. Display also the ingredients of those pizzas on hover. Display the total number very big in the middle of each bar.
    </input>
    <output>
        worst_sold = df.groupby('pizza_name', as_index=False)['quantity'].sum().nsmallest(5, 'quantity')
        fig = go.Figure(
        data=[
            go.Bar(
                x=worst_sold['pizza_name'],
                y=worst_sold['quantity'],
                text=worst_sold['quantity'],
                textposition='inside',
                textfont=dict(size=24),
                hovertext=worst_sold['pizza_name'].map(
                    df.drop_duplicates('pizza_name').set_index('pizza_name')['pizza_ingredients']
                ),
                hoverinfo='text'
                )
            ]
        ) 
    </output>

## Example 2:
    <input>
        Show orders per hour (order_time) throughout the day.
    </input>
    <output>
        df['order_time'] = pd.to_datetime(df['order_time'], format='%H:%M:%S').dt.hour
        orders_per_hour = df.groupby('order_time', as_index=False)['order_id'].count()
        fig = go.Figure(
            data=[
                go.Bar(
                    x=orders_per_hour['order_time'],
                    y=orders_per_hour['order_id'],
                    text=orders_per_hour['order_id'],
                    textposition='outside'
                )
            ]
        )
    </output>
"""


def generate_plot(df: pd.DataFrame, user_input: str) -> str: 
    """
    Generates a plotly figure based on the provided DataFrame and user input.
    
    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        user_input (str): The user input describing the desired plot.
    
    Returns:
        Response: the python code generated by the OpenAI API as str
    """

    # Capture df.info() output as a string
    buf = io.StringIO()
    df.info(buf=buf)
    df_info_str = buf.getvalue()

    # Create the response using the Responses API
    response = openai_client.responses.create(
        model="gpt-4o",
        input=[
            {
                "role": "developer",
                "content": [
                    {"type": "input_text", "text": system_prompt},
                ],
            },
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": user_input},
                ],
            },
        ],
    )
    code = response.output_text
    return code

In [ ]:
import plotly.graph_objects as go

user_input = "Show a bar plot of the top 10 most sold pizzas based on pizza name. Display also the ingredients of those pizzas."
# user_input = "Show pizza ingredients as a pie chart. Only include ingredients that appear in more than 2% of total sales."
# user_input = "Create a time series line chart of total sales per month over the entire dataset."
# user_input = "Plot total revenue per pizza category as a horizontal bar chart, sorted by revenue."
# user_input = "Display a stacked bar chart of pizza quantities sold per category, broken down by pizza size."
# user_input = "Generate a line chart showing the cumulative revenue over time for the top 3 most popular pizza categories."
# user_input = "Show orders per hour throughout the day."
# user_input = "Create a heatmap of average quantity sold by pizza size and pizza category."
# user_input = "Compare monthly sales trends of 'The Hawaiian Pizza' and 'The Classic Deluxe Pizza' using a dual-line chart with date on the x-axis."

code = generate_plot(df, user_input)
print(code)
fig = None
exec(code)
fig.show()